In [1]:
search_for = 134
start_from = 0
threads = 10
thread = 9
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.05135178565979004
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 634149671
type: [1, 1, 1, 1, 134] 134
cases of this type: 322417936
100000 11.693008579718878
200000 19.06256193953331
300000 26.043719559804746
400000 26.39783484004958
500000 26.365825328777124
600000 27.107707106368014
700000 32.18671801163162
800000 35.170331545831225
900000 38.48238561864496
1000000 45.91510154092414
1100000 48.59001205784546
1200000 35.23492599216537
1300000 25.85477138688905
1400000 46.89705996529904
1500000 47.15945522148752
1600000 47.56922875981859
1700000 47.41228330667796
1800000 50.22936530341915
1900000 49.8366211422286
2000000 47.86173957027461
2100000 47.409743392810256
2200000 47.84707624590084
2300000 48.31624223686884
2400000 39.63072820657778
2500000 19.91339420213807
2600000 46.958659710014096
2700000 47.17383443415477
2800000 46.61073691225452
2900000 46.213503922982866
3000000 46.3469942486

29800000 47.829245710156606
29900000 46.29646324278072
30000000 45.56268523890439
30100000 23.613559541102667
30200000 36.40708122099972
30300000 45.728446183071696
30400000 46.89477755748236
30500000 47.974112621886476
30600000 47.46928097095291
30700000 47.53864691036591
30800000 49.33240476709065
30900000 49.33933326554498
31000000 49.14677250305533
31100000 23.920711034847667
31200000 45.387917740549526
31300000 51.35127344667977
31400000 51.688799941836756
31500000 44.18299114669882
31600000 25.320311800019443
31700000 50.37038342213392
31800000 51.577580098996286
31900000 51.43473552475286
32000000 51.78122246109855
32100000 49.92269612934461
32200000 48.52933402379763
32300000 48.05829260501894
32400000 46.94493525355563
32500000 26.2270937353511
32600000 34.354612655263125
32700000 46.36725987560112
32800000 48.64782526732205
32900000 48.603440300225635
33000000 50.27408142748647
33100000 48.46147528729308
33200000 51.01099186804967
33300000 50.98485898967871
33400000 50.066656

59800000 23.47215958382014
59900000 46.62141971530097
60000000 45.55004710107135
60100000 44.60382767006019
60200000 47.418393808791336
60300000 49.09894152158234
60400000 49.51061188804352
60500000 49.553913293008875
60600000 37.83986193997622
60700000 28.19738737580014
60800000 47.463058459185696
60900000 46.45522258549467
61000000 47.23807241965638
61100000 44.74113484318071
61200000 44.15012620083955
61300000 42.4629984919404
61400000 14.187702124531306
61500000 41.567832249781674
61600000 43.64313784829933
61700000 44.61738385681586
61800000 43.44017425534684
61900000 45.80812998873531
62000000 45.596299585508724
62100000 38.70671927420131
62200000 29.76886206210828
62300000 49.6997117437281
62400000 48.49596934751811
62500000 48.20070076254643
62600000 48.912726843996005
62700000 47.08413047824495
62800000 46.72515802305007
62900000 48.0460584166416
63000000 45.48056776798893
63100000 19.34191279221611
63200000 44.61200389419815
63300000 44.02845395240399
63400000 42.516707585083

89900000 40.14953569310378
90000000 43.17924073719517
90100000 40.08170868411534
90200000 39.92148461876494
90300000 12.396613659396879
90400000 40.87448367678199
90500000 41.593020210859876
90600000 39.110715936613495
90700000 40.68732048545912
90800000 21.844759540185784
90900000 37.1234343479501
91000000 43.712515402194605
91100000 43.27882739272646
91200000 46.06853957166738
91300000 44.868835115212576
91400000 43.08241406420581
91500000 44.68812388232545
91600000 43.780133375034474
91700000 41.99189764511472
91800000 42.90791270593791
91900000 40.749620552285975
92000000 41.51771835697389
92100000 33.93737434847184
92200000 18.993276177864352
92300000 38.12555531958589
92400000 37.16762800368697
92500000 39.28687052163309
92600000 40.121888330123085
92700000 9.569685689993651
92800000 40.67388082721149
92900000 37.550235303172656
93000000 38.624523978685026
93100000 37.11397481306618
93200000 18.23333099784599
93300000 37.95344117741304
93400000 41.418532907179255
93500000 40.4823

119200000 21.703979712135922
119300000 37.33385961163532
119400000 35.36198836892043
119500000 18.764634315439988
119600000 37.23218530475194
119700000 38.325987290031264
119800000 36.73820063303806
119900000 39.475829454077974
120000000 41.179059040471365
120100000 41.89792513628167
120200000 42.6301697363386
120300000 42.23582292663541
120400000 39.94152253836533
120500000 41.2832883275561
120600000 40.221720350594346
120700000 38.70959322201122
120800000 37.49636307253595
120900000 38.297590762930334
121000000 36.17927659791306
121100000 36.635219388914756
121200000 28.40964390081064
121300000 22.31236101605154
121400000 35.44057833829372
121500000 28.538318564949982
121600000 19.308805387100005
121700000 34.63329710560852
121800000 31.49271998214225
121900000 21.066855395088865
122000000 36.75686374713324
122100000 37.446379108185674
122200000 35.95087958230121
122300000 37.39028267107509
122400000 37.84510485773518
122500000 40.024021547359894
122600000 40.14182862933029
122700000

...

493800000 7.677645815915887
493900000 3.739945119204496
494000000 8.365277771787865
494100000 7.462868296067273
494200000 7.144582623429045
494300000 7.449231012910903
494400000 6.80838409247215
494500000 5.157458437755414
494600000 6.158513042548645
494700000 7.78868662360802
494800000 7.686688238090527
494900000 7.6328112139880515
495000000 7.846704234319328
495100000 3.5003001995311496
495200000 8.06556350344964
495300000 7.63689290244637
495400000 7.395307553912733
495500000 7.3983075002913585
495600000 6.316570068649135
495700000 6.291879838745104
495800000 5.683615956465934
495900000 7.850921236467354
496000000 7.513234331149691
496100000 7.7477851436893
496200000 7.514057094074741
496300000 3.74903296449936
496400000 7.493599295594228
496500000 7.7603357982902725
496600000 7.1729211681627
496700000 7.188658327563915
496800000 5.459301102736612
496900000 6.707204253053048
497000000 5.708026549160219
497100000 7.909112444285755
497200000 7.347786700940321
497300000 7.5603926859127

523100000 5.682017473724327
523200000 5.621555001854443
523300000 5.681282556655178
523400000 5.9161178151423215
523500000 5.77427334122588
523600000 5.671643171228592
523700000 5.583243290153141
523800000 5.858235377590774
523900000 4.272774076846956
524000000 2.749233292837304
524100000 3.400185892170105
524200000 5.306468872177653
524300000 5.708108505481206
524400000 5.45142630507021
524500000 5.693425573175638
524600000 5.520363452490711
524700000 5.4980298136218275
524800000 5.3727130275119
524900000 5.204497007318475
525000000 5.775411406090059
525100000 4.166051919918965
525200000 2.848280036657274
525300000 3.439668685081639
525400000 5.0674336953202985
525500000 5.112448034361883
525600000 5.306435039874931
525700000 5.348428690369535
525800000 5.647626961764459
525900000 5.859811259627529
526000000 5.574505395614851
526100000 5.629332843231219
526200000 5.468215257678835
526300000 4.334607344755812
526400000 1.9233278093159356
526500000 4.07915357453158
526600000 5.530880442

552200000 4.830987786272246
552300000 5.008301183829318
552400000 4.52504068490582
552500000 3.298811939557872
552600000 3.8101346863188477
552700000 4.2635605195664965
552800000 3.7546198024874244
552900000 2.7445538842761295
553000000 4.158621621758074
553100000 4.449929823899108
553200000 3.1678999514220956
553300000 4.5433276762505725
553400000 4.7164212812947595
553500000 4.785344969446407
553600000 4.391236226169688
553700000 2.8876766772845444
553800000 4.237284561283345
553900000 4.081411875215777
554000000 3.753049029697961
554100000 2.79838405821002
554200000 4.08050220300025
554300000 4.249305137066545
554400000 3.2359678886944154
554500000 4.520856234770556
554600000 4.559418361288735
554700000 4.658425255290506
554800000 4.592104401294044
554900000 2.556421766793251
555000000 4.362564424965412
555100000 4.188635230580417
555200000 3.7157141155929585
555300000 2.4598095988643998
555400000 3.930025184903552
555500000 4.189951764769226
555600000 3.0077762485585517
555700000 4

579800000 1.9796130010514714
579900000 2.054748862530735
580000000 2.5808061582380706
580100000 2.487043511107516
580200000 2.075396360467438
580300000 1.979435432079773
580400000 1.8219507764710683
580500000 1.6799199675509557
580600000 2.669455901816928
580700000 2.736968489583371
580800000 1.852631795800096
580900000 1.9207985676732249
581000000 2.0510809722126244
581100000 1.8335205928495533
581200000 2.764656048231869
581300000 2.4803366601274606
581400000 1.8125801544465336
581500000 1.841846046681375
581600000 1.9906376809505344
581700000 1.7569106105267083
581800000 2.8617366762533747
581900000 2.9444181388633135
582000000 1.725850234360356
582100000 1.9831619360193407
582200000 1.9897903547367335
582300000 1.6236962195663247
582400000 2.658503475171095
582500000 2.7340186681329586
582600000 1.7229639042472071
582700000 1.9750914346268944
582800000 1.6947067825046718
582900000 1.6821281900294816
583000000 2.710650542047716
583100000 2.5187488992741085
583200000 1.64826891486696

608300000 1.0786236359974504
608400000 0.9997680503850719
608500000 1.2558824896357035
608600000 0.5355952300886253
608700000 1.1009415460647722
608800000 1.0088278144703646
608900000 0.9866502357523979
609000000 0.6541551193159395
609100000 0.6664575316017005
609200000 0.6736010164138794
609300000 1.0489436683161186
609400000 1.1365636987730436
609500000 0.7732573398462534
609600000 0.8410991760022614
609700000 1.0297380404829979
609800000 0.5347299392548681
609900000 0.7081373741264337
610000000 1.026762903872403
610100000 0.8333287278780123
610200000 0.6435394850733394
610300000 0.6644617786254425
610400000 0.39570134337746504
610500000 0.6891249482035816
610600000 1.146713798450462
610700000 0.7096106649025024
610800000 0.9657752403317181
610900000 0.8070512954173492
611000000 0.39187493051317535
611100000 0.7492100843796472
611200000 1.1148237979355904
611300000 0.4973547233740091
611400000 0.8915495939853681
611500000 0.9769825169596693
611600000 0.7730631928158045
611700000 0.88